In [29]:
import pandas as pd
from config import api_key
import requests

In [30]:
url = f'https://api.pubg.com/shards/console/samples'
header= {"Authorization": f"Bearer {api_key}","Accept": "application/vnd.api+json"}
response = requests.get(url,headers=header)
r = response.json()
matches =r['data']['relationships']['matches']['data']
match_name = pd.json_normalize(matches)
match_name

,type,id
0,match,b62ce27e-d6c8-47d7-bc0b-d7c16858074f
1,match,920cdf19-adec-4f0a-99e3-6e03dde7aa5c
2,match,02e8332c-55aa-417f-9615-8794342a0d9c
3,match,cc15050c-9b83-4cfb-a575-2b328a66646c
4,match,a75db296-5b70-4b36-8493-2b97f5543759
...,...,...
230,match,d8ff7c4c-323f-4aec-9155-a1cbb8c2fe98
231,match,e9f45141-656b-4575-8231-c7dbdddc93bd
232,match,f0ffa98c-f78e-4b1f-9530-eee2036248ba
233,match,d7fd3775-b451-4109-ad5c-d88b3007e58e


In [31]:
duration_list = []
map_list =[]
game_mode = []
match_type = []
match_list = []
stats_list = []
for match in match_name['id']:
    match_id = match
    url = f'https://api.pubg.com/shards/console/matches/{match_id}'
    header= {"Authorization": f"Bearer {api_key}","Accept": "application/vnd.api+json"}
    response = requests.get(url,headers=header)
    r = response.json()
    length = len(r['included'])-1
    for x in range(length):
        match_list.append(match_id)
        map_list.append(r['data']['attributes']['mapName'])
        duration_list.append(r['data']['attributes']['duration'])
        game_mode.append(r['data']['attributes']['gameMode'])
        match_type.append(r['data']['attributes']['matchType'])
        df =f'{x}'
        stats = r['included'][x]['attributes']
        df = pd.json_normalize(stats)
        stats_list.append(df)


In [32]:
new_df = pd.concat(stats_list)
new_df.drop(['actor', 'shardId','won', 'createdAt', 'URL', 'name', 'description',
             'stats.teamId'], axis=1, inplace=True)
new_df.insert(0, 'match_id', match_list)
new_df.insert(1, 'game_duration', duration_list)
new_df.insert(2, 'game_mode', game_mode)
new_df.insert(3, 'map', map_list)
new_df.insert(4, 'match_type', match_type)

In [33]:
new_df.to_csv('model_samps.csv')